## 設定ファイルの読み込み
OpanAIのAPIキーを読み込んでいる。

In [51]:
from dotenv import load_dotenv

load_dotenv()

True

## Token数を調べる
tiktokenを使ってtoken数を調べてみる。

In [70]:
!pip install tiktoken


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [52]:
import tiktoken

tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')
tokenizer

<Encoding 'cl100k_base'>

ChatGPTでは `cl100k_base` というtokenizerが使われている。

実際に `cl100k_base` tokenizerで英文をエンコーディングする。

In [7]:
tokenizer.encode("this is a pen.")

[576, 374, 264, 5869, 13]

単語及びピリオドが数値にmappingされていることがわかる。
例えばthisの数値を調べてみると

In [8]:
tokenizer.encode("this")

[576]

となる。このように、それぞれの単語と対応する数値がある。

token数はmappingされたベクターの長さを調べれば良い。

In [9]:
len(tokenizer.encode("this is a pen."))

5

次に、日本語をエンコーディングしてみる。

In [10]:
tokenizer.encode("こんにちは")

[90115]

In [11]:
len(tokenizer.encode("こんにちは"))

1

「こんにちは」は、token数1となることがわかる。

一方で「こんばんは」はどうなるか。

In [12]:
tokenizer.encode("こんばんは")

[69847, 81219, 25827, 15682]

In [13]:
len(tokenizer.encode("こんばんは"))

4

「こんばんは」はtoken数が4となることがわかる。

このように日本語では、token数は単語によって大きく変わってくる。

## Document Loader

### PDFファイルを読み込むLoader
PDFを読むことができるPyPDFLoaderはpypdfをインストールする必要がある。

In [55]:
!pip install pypdf


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [56]:
from langchain.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader("https://arxiv.org/pdf/2210.03629.pdf")

In [57]:
arxiv_by_pdf_loader = pdf_loader.load()
arxiv_by_pdf_loader

[Document(page_content='Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and update action plans as well as handle\nexceptions, whi

In [16]:
len(arxiv_by_pdf_loader)

33

1ページごとPDFのコンテンツが読み込まれている。

読み込まれたコンテンツを確認するためには、 `page_content` を利用する。

In [58]:
arxiv_by_pdf_loader[0].page_content

'Published as a conference paper at ICLR 2023\nREAC T: S YNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and update action plans as well as handle\nexceptions, while actions allow it to 

### arXivから論文を取得するLoader
ArxivLoaderはarxivとpymupdfをインストールする必要がある。

In [60]:
!pip install arxiv pymupdf


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [61]:
from langchain.document_loaders import ArxivLoader
arxiv_loader = ArxivLoader("2210.03629")

In [62]:
arxiv_by_arxiv_loader = arxiv_loader.load()

In [63]:
len(arxiv_by_arxiv_loader)

1

ArxivLoaderでは１つのDocumentとして読み込まれる

In [64]:
arxiv_by_arxiv_loader[0].page_content

'Published as a conference paper at ICLR 2023\nREACT: SYNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and update action plans as well as handle\nexceptions, while actions allow it to in

分割したい場合は `load_and_split` を利用。

In [65]:
arxiv_loader.load_and_split()

[Document(page_content='Published as a conference paper at ICLR 2023\nREACT: SYNERGIZING REASONING AND ACTING IN\nLANGUAGE MODELS\nShunyu Yao∗*,1, Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1, Yuan Cao2\n1Department of Computer Science, Princeton University\n2Google Research, Brain team\n1{shunyuy,karthikn}@princeton.edu\n2{jeffreyzhao,dianyu,dunan,izhak,yuancao}@google.com\nABSTRACT\nWhile large language models (LLMs) have demonstrated impressive performance\nacross tasks in language understanding and interactive decision making, their\nabilities for reasoning (e.g. chain-of-thought prompting) and acting (e.g. action\nplan generation) have primarily been studied as separate topics. In this paper, we\nexplore the use of LLMs to generate both reasoning traces and task-speciﬁc actions\nin an interleaved manner, allowing for greater synergy between the two: reasoning\ntraces help the model induce, track, and update action plans as well as handle\nexceptions, while

### Wikipediaの検索結果を取得するLoader
WikipediaLoaderは、wikipediaをインストールする必要がある。

In [66]:
!pip install wikipedia


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [68]:
from langchain.document_loaders import WikipediaLoader

wikipedia_loader = WikipediaLoader(query="ChatGPT", lang="ja", load_max_docs=2)

In [71]:
wiki = wikipedia_loader.load()
wiki

[Document(page_content='ChatGPT（チャットジーピーティー、英語: Chat Generative Pre-trained Transformer）は、OpenAIが2022年11月に公開した人工知能チャットボットであり、生成AIの一種。\nGPTの原語のGenerative Pre-trained Transformerとは、「生成可能な事前学習済み変換器」という意味である。OpenAIのGPT-3ファミリーの大規模な言語モデルに基づいて構築されており、教師あり学習と強化学習の両方の手法を使って転移学習され、機械学習のサブセットである深層学習を使って開発されている。\n\n\n== 概要 ==\n2022年11月30日にプロトタイプとして公開され、幅広い分野の質問に詳細な回答を生成できることから注目を集めた。しかし、人間が自然と感じる回答の生成を特徴としていることから、一見自然に見えるが事実とは異なる回答を生成することもあり、大きな欠点とされた。ChatGPTのリリース後、OpenAIの評価額は290億米ドルとなり、2021年時の140億ドルと比べて2倍以上に増加した。OpenAIの筆頭株主であるMicrosoftが展開する検索エンジンのBingやウェブブラウザのEdge等の幅広いサービスの生成AIの技術にも、ChatGTが活用されている。\nスイスの金融グループUBSの分析によると、ChatGPTがアクティブユーザー数1億人に到達するのにかかった時間は、TikTokとInstagramがそれぞれ9カ月と2年半であるのに対して、2か月と史上最速である。\nOpenAIによれば、オプトアウトを申請しない限り、ユーザーが入力したChatGPTとの会話内容はOpenAIによってその内容を利用されることがある。このため、一般的にChatGPTの利用に際して個人情報と機密情報の入力に注意する必要があり、Apple、JPモルガン、サムスン電子など一部の企業はChatGPTへのアクセスを禁止している。\nChatGPTによる個人情報の収集に関しては、EU一般データ保護規則に違反している疑惑も浮上しており、2023年3月31日、イタリア政府はChatGPTの訓練のために個人情報を大量に収集することを正当化する法的根拠がないと指摘して、ChatGPTへの

### ウェブページを取得
WebBaseLoaderを使うことでウェブページを読み込みができる。

In [75]:
from langchain.document_loaders import WebBaseLoader

web = WebBaseLoader("https://ja.wikipedia.org/wiki/ChatGPT").load()

In [76]:
web[0].page_content

'\n\n\n\nChatGPT - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nコンテンツにスキップ\n\n\n\n\n\n\n\nメインメニュー\n\n\n\n\n\nメインメニュー\nサイドバーに移動\n非表示\n\n\n\n\t\t案内\n\t\n\n\nメインページコミュニティ・ポータル最近の出来事新しいページ最近の更新おまかせ表示練習用ページアップロード (ウィキメディア・コモンズ)\n\n\n\n\n\n\t\tヘルプ\n\t\n\n\nヘルプ井戸端お知らせバグの報告寄付ウィキペディアに関するお問い合わせ\n\n\n\n\n\n言語\n\n言語間リンクはページの先頭にあるページ名の向かい側に設置されています。\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n検索\n\n\n\n\n\n\n\n\n\n\n\n検索\n\n\n\n\n\n\n\n\nアカウント作成ログイン\n\n\n\n\n\n\n個人用ツール\n\n\n\n\n\n アカウント作成 ログイン\n\n\n\n\n\n\t\tログアウトした編集者のページ もっと詳しく\n\n\n\n投稿記録トーク\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n目次\nサイドバーに移動\n非表示\n\n\n\n\nページ先頭\n\n\n\n\n\n1概要\n\n\n\n\n\n\n\n2トレーニング\n\n\n\n\n\n\n\n3機能\n\n\n\n\n\n\n\n4サービス\n\n\n\n\n\n\n\n5反響\n\n\n\n\n\n\n\n6批判\n\n\n\n\n\n\n\n7学術界の反応\n\n\n\n学術界の反応サブセクションを切り替えます\n\n\n\n\n\n7.1東京大学\n\n\n\n\n\n\n\n\n\n8問題\n\n\n\n問題サブセクションを切り替えます\n\n\n\n\n\n8.1スパム・誤情報への悪用\n\n\n\n\n\n\n\n8.2機密情報の流出\n\n\n\n\n\n\n\n8.3データ倫理・著作権\n\n\n\n\n\n\n\n8.4個人情報保護\n\n

### YouTubeの文字起こしを取得
YouTubeのtranscriptを取得するためにはyoutube-transcript-apiをインストールする必要がある。

In [77]:
!pip install youtube-transcript-api


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [78]:
from langchain.document_loaders import YoutubeLoader
youtube = YoutubeLoader.from_youtube_url(
   youtube_url="https://www.youtube.com/watch?v=I2UZutRrtEE",
   language="ja"
).load()

In [79]:
youtube

[Document(page_content='では早速なんですけれども始めさせて いただきます私からはラングチェイン エージェントを使って自社ツールと チャットGPTを連携するというタイトル で発表させていただきますChatGPT 自体は皆さんご存知のように非常に優れた 能力を持っていますが最新のデータに アクセスできないとか特定の数学的な推論 が得意でないなどいくつかの課題が存在し ますこれらの課題は自社プロダクトに チャートGPTを導入するなどチャット GPTを社会実装しようとした際に解決し なければなりません今回お伝えするラグ チェインエイジェントを導入すると ChatGPTと外部ツールをうまく連携 させることができるようになりChat GPTが抱える課題を克服しチットGPT の可能性をさらに広げることができます 本日はラングチェインエージェントをどう 活用することでチャットGPTの能力を 拡張できるのか具体的にご紹介していき ますそれでは始めていきますこちらが私の プロフィールとなります内容はいつもと 同じなのでスキップさせていただきます はいでは具体的な話に入る前にですね今回 のテーマの結論に先に触れておきます結論 なんですけれどもラグチェインには様々な エージェントの種類が存在するが自社 ツールとチャットGPTを効果的に連結 する際汎用的なエージェントタイプである ゼロショットリアクトまたはオAI ファンクションを利用することが簡単と なりますゼロショットリアクトとOpen AIファンクションを比較するとOpen AIファンクションの方がチットGPT からより適切かつ精度の高い答えを 引き出しやすくなっていますこちらはデモ のでお見せしますねでそしてえっとここで ChatGPTと自社ツール連携で非常に 重要となるものとしてツールの説明という ものがありますと通常のプロンプトチャト GPTを使う時のプロンプトと同様に ツールの特性や機能をChatGPTに 正確かつ効果的に伝えることで自社ツール とチャットGPTの連携の成功率が格段に 向上していきますこのポイントに着目し ながら具体的な方法論について今日は 深掘りしていきたいと思いますそれでは これらのポイントを年頭にきつつ今回の 発表をお聞きください本日の発表は以下の 3つのセクションから構成されていま

## Summarize Chain
Document Loaderで取得したDocumentをSummarize Chainを使って要約する。

In [80]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")
summarize_chain = load_summarize_chain(llm=llm, chain_type="refine", verbose=True)

In [81]:
summarize_chain(youtube)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"では早速なんですけれども始めさせて いただきます私からはラングチェイン エージェントを使って自社ツールと チャットGPTを連携するというタイトル で発表させていただきますChatGPT 自体は皆さんご存知のように非常に優れた 能力を持っていますが最新のデータに アクセスできないとか特定の数学的な推論 が得意でないなどいくつかの課題が存在し ますこれらの課題は自社プロダクトに チャートGPTを導入するなどチャット GPTを社会実装しようとした際に解決し なければなりません今回お伝えするラグ チェインエイジェントを導入すると ChatGPTと外部ツールをうまく連携 させることができるようになりChat GPTが抱える課題を克服しチットGPT の可能性をさらに広げることができます 本日はラングチェインエージェントをどう 活用することでチャットGPTの能力を 拡張できるのか具体的にご紹介していき ますそれでは始めていきますこちらが私の プロフィールとなります内容はいつもと 同じなのでスキップさせていただきます はいでは具体的な話に入る前にですね今回 のテーマの結論に先に触れておきます結論 なんですけれどもラグチェインには様々な エージェントの種類が存在するが自社 ツールとチャットGPTを効果的に連結 する際汎用的なエージェントタイプである ゼロショットリアクトまたはオAI ファンクションを利用することが簡単と なりますゼロショットリアクトとOpen AIファンクションを比較するとOpen AIファンクションの方がチットGPT からより適切かつ精度の高い答えを 引き出しやすくなっていますこちらはデモ のでお見せしますねでそしてえっとここで ChatGPTと自社ツール連携で非常に 重要となるものとしてツールの説明という ものがありますと通常のプロンプトチャト GPTを使う時のプロンプトと同様に ツールの特性や機能をChatGPTに 正確かつ効果的に伝えることで自社ツール とチャッ

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 19305 tokens. Please reduce the length of the messages.

もとの文書が長すぎると、token数制限に引っかかりエラーとなってしまう。

長い文書はload_and_splitを使ってドキュメントを分割する。

In [82]:
split_youtube = YoutubeLoader.from_youtube_url(
   youtube_url="https://www.youtube.com/watch?v=I2UZutRrtEE",
   language="ja"
).load_and_split()

In [83]:
%time summary_output = summarize_chain(split_youtube)["output_text"]



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"では早速なんですけれども始めさせて いただきます私からはラングチェイン エージェントを使って自社ツールと チャットGPTを連携するというタイトル で発表させていただきますChatGPT 自体は皆さんご存知のように非常に優れた 能力を持っていますが最新のデータに アクセスできないとか特定の数学的な推論 が得意でないなどいくつかの課題が存在し ますこれらの課題は自社プロダクトに チャートGPTを導入するなどチャット GPTを社会実装しようとした際に解決し なければなりません今回お伝えするラグ チェインエイジェントを導入すると ChatGPTと外部ツールをうまく連携 させることができるようになりChat GPTが抱える課題を克服しチットGPT の可能性をさらに広げることができます 本日はラングチェインエージェントをどう 活用することでチャットGPTの能力を 拡張できるのか具体的にご紹介していき ますそれでは始めていきますこちらが私の プロフィールとなります内容はいつもと 同じなのでスキップさせていただきます はいでは具体的な話に入る前にですね今回 のテーマの結論に先に触れておきます結論 なんですけれどもラグチェインには様々な エージェントの種類が存在するが自社 ツールとチャットGPTを効果的に連結 する際汎用的なエージェントタイプである ゼロショットリアクトまたはオAI ファンクションを利用することが簡単と なりますゼロショットリアクトとOpen AIファンクションを比較するとOpen AIファンクションの方がチットGPT からより適切かつ精度の高い答えを 引き出しやすくなっていますこちらはデモ のでお見せしますねでそしてえっとここで ChatGPTと自社ツール連携で非常に 重要となるものとしてツールの説明という ものがありますと通常のプロンプトチャト GPTを使う時のプロンプトと同様に ツールの特性や機能をChatGPTに 正確かつ効果的に伝えることで自社ツール とチャッ

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: The speaker introduces the topic of integrating the company's tool with ChatGPT using the LangChain Agent. They highlight the challenges faced by ChatGPT, such as limited access to the latest data and specific mathematical inference. The LangChain Agent aims to resolve these challenges and expand the potential of ChatGPT. The presentation will cover the basics of ChatGPT, the features of LangChain Agent, and a demonstration. They also discuss the importance of providing accurate information about the tool to improve the success rate of integrating ChatGPT with external tools. The speaker mentions the growing use of ChatGPT in various business sectors and the need for good prompt engineering techniques to obtain accurate answers. They briefly explain the basics of prompt engineering and mention the Fusion Probi

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).



> Finished chain.

> Finished chain.
CPU times: user 222 ms, sys: 45.1 ms, total: 267 ms
Wall time: 2h 18min 35s


In [84]:
summary_output

'The LangChain Agent is introduced as a framework to enhance ChatGPT\'s capabilities. OpenAI\'s Function Calling feature is used to integrate ChatGPT with external tools. The LangChain Agent interacts with ChatGPT to determine appropriate clothing for specific weather conditions. The use of Pathetic is discussed to provide additional information and context to ChatGPT\'s responses. The introduction of a new tool called "GetDate" is proposed to obtain the current date and improve the accuracy of recommendations. The decorator function is also mentioned as a simpler alternative to create tools. However, it is noted that passing incorrect or insufficient information may lead to inaccurate responses. The addition of a tool called "SaveData" is suggested to write data to a database. Examples of using the "SaveData" tool to write date, item name, and item price are provided. The potential use of SQL statements for database operations is mentioned. The demonstration showcases the capability t

## QA Chain
Document Loaderで取得したDocumentにQA Chainを使って質問をする。

In [89]:
from langchain.chains.question_answering import load_qa_chain

qa_chain = load_qa_chain(llm=llm, chain_type="map_reduce", verbose=True)

In [90]:
%%time
qa_output = qa_chain(
    {
        "input_documents": split_youtube,
        "question": "ChatGPTが不得意なことを教えて",
    },
    return_only_outputs=True,
)["output_text"]



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
______________________
では早速なんですけれども始めさせて いただきます私からはラングチェイン エージェントを使って自社ツールと チャットGPTを連携するというタイトル で発表させていただきますChatGPT 自体は皆さんご存知のように非常に優れた 能力を持っていますが最新のデータに アクセスできないとか特定の数学的な推論 が得意でないなどいくつかの課題が存在し ますこれらの課題は自社プロダクトに チャートGPTを導入するなどチャット GPTを社会実装しようとした際に解決し なければなりません今回お伝えするラグ チェインエイジェントを導入すると ChatGPTと外部ツールをうまく連携 させることができるようになりChat GPTが抱える課題を克服しチットGPT の可能性をさらに広げることができます 本日はラングチェインエージェントをどう 活用することでチャットGPTの能力を 拡張できるのか具体的にご紹介していき ますそれでは始めていきますこちらが私の プロフィールとなります内容はいつもと 同じなのでスキップさせていただきます はいでは具体的な話に入る前にですね今回 のテーマの結論に先に触れておきます結論 なんですけれどもラグチェインには様々な エージェントの種類が存在するが自社 ツールとチャットGPTを効果的に連結 する際汎用的なエージェントタイプである ゼロショットリアクトまたはオAI ファンクションを利用することが簡単と なりますゼロショットリアクトとOpen AIファンクションを比較するとOpen AIファンクションの方がチットGPT からより適切かつ精度の高い答えを 引き出しやすくなっていますこちらはデモ のでお見せしますね

In [91]:
print(qa_output)

ChatGPTが不得意とされることは以下の通りです：

1. 最新の情報へのアクセス：ChatGPTはリアルタイムで変化する情報には対応できません。例えば、為替や株価のような情報に関しては最新のデータにアクセスすることができません。

2. 独自の情報源へのアクセス：ChatGPTは特定のプライベートな情報源や企業のデータベースへのアクセスはできません。

3. 正確性の課題：ChatGPTは時折簡単な算術計算ですら間違えることがあり、正確な情報や結果が求められるタスクにおいて問題となることがあります。

4. データの不足への対応：ChatGPTはファインチューニングによる性能改善は可能ですが、特定のタスクに特化したファインチューニングは他の多様なタスクへの性能低下をもたらす可能性があります。

5. 文脈の理解：ChatGPTは単一の入力文を受け取るため、前後の文脈や会話の流れを理解するのが難しい場合があります。そのため、一部の質問や指示に対して適切な回答をすることができないことがあります。

6. ユーザーの意図の理解：ChatGPTはユーザーの意図を完全に理解することができない場合があります。そのため、質問や指示を明確にすることが重要です。

7. ファクトチェックや正確性の保証ができない：ChatGPTはトレーニングデータから学習した情報を元に応答を生成するため、正確性や信頼性の保証はされていません。

8. 長文への対応：ChatGPTは一度に生成できるテキストの長さに制限があり、長文や複雑な文章に対しては適切な応答ができないことがあります。

これらの制約を考慮しながら、ChatGPTを利用する際は注意が必要です。
